In [ ]:
pip install pickle5

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy 
import pickle
from glob import glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from glob import glob
import pickle5 as pickle
import numpy as np

ROOT_PATH = "../input/"

cities = ["austin", "miami", "pittsburgh", "dearborn", "washington-dc", "palo-alto"]
splits = ["train", "test"]

def get_city_trajectories(city="palo-alto", split="train", normalized=False):
    if split=="train":
        f_in = ROOT_PATH + split + "/" + city + "_inputs"
        inputs = pickle.load(open(f_in, "rb"))
        inputs = np.asarray(inputs)

        outputs = None
    
        f_out = ROOT_PATH + split + "/" + city + "_outputs"
        outputs = pickle.load(open(f_out, "rb"))
        outputs = np.asarray(outputs)
        
    elif split=="test":
        f_in = ROOT_PATH + "test-inputs" + "/" + city + "_inputs"
        inputs = pickle.load(open(f_in, "rb"))
        inputs = np.asarray(inputs)

        outputs = None
        
    return inputs, outputs

In [ ]:
p_i, p_o = get_city_trajectories(city="palo-alto", split="train", normalized=True)

In [ ]:

train_combined = np.concatenate([p_i, p_o], axis=1)

In [ ]:

p_test_i, p_test_o = get_city_trajectories(city="palo-alto", split="test", normalized=True)

In [ ]:

def preprocess_data(inp, x_or_y=0):
    
    final_feature = inp[:, :50, x_or_y]
    final_target = inp[:, 50, x_or_y]
    for i in range(1, 60):
        temp_x = inp[:, i:50+i, x_or_y]
        temp_y = inp[:, 50+i, x_or_y]
        final_feature = np.concatenate([final_feature, temp_x], axis=0)
        final_target = np.concatenate([final_target, temp_y], axis=0)
    
    return final_feature, final_target

In [ ]:


X, y = preprocess_data(train_combined, 1)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
reg = LinearRegression().fit(X, y)

In [ ]:
reg1 = LinearRegression().fit(X, y)

In [ ]:
def predict_regressor(model, inp, pred_len, x_or_y=0):
    
    result = []
    curr = inp[:, x_or_y]
    pred = model.predict(curr.reshape(1, -1))[0]
    result.append(pred)
    for i in range(1, pred_len):
        curr = np.append(curr[1:], pred)
        pred = model.predict(curr.reshape(1, -1))[0]
        result.append(pred)
    
    return result

In [ ]:
p_test_i.shape

In [ ]:
palo_df = pd.DataFrame()
for i in range(len(p_test_i)):
    
    pred_x = predict_regressor(reg, p_test_i[i], 60, 0)
    pred_y = predict_regressor(reg1, p_test_i[i], 60, 1)
    print("predicts for " + str(i) + " is done")
    temp_dict = {}
    for j in range(0, 120, 2):
        col_x = 'v' + str(j)
        col_y = 'v' + str(j+1)
        row = j // 2
        temp_dict[col_x] = pred_x[row]
        temp_dict[col_y] = pred_y[row]
    
    temp_idx_name = str(i) + "_" + "palo-alto"
    temp_final = pd.DataFrame(temp_dict, index=[temp_idx_name])
    palo_df = pd.concat([palo_df, temp_final])

In [ ]:
palo_df.reset_index(level=0).rename(columns={"index": "ID"}).to_csv("model3_palo_df.csv", index=False)

In [ ]:
df_actual_path = "../input/pred-actual/palo-alto_actual.csv"
df_pred_path = "./model3_palo_df.csv"

def actual_error(df_actual_path, df_pred_path):
    actual = pd.read_csv(df_actual_path)    
    actual_array = actual.iloc[:, 1:].to_numpy()
    
    pred1 = pd.read_csv(df_pred_path)
    pred1_array = pred1.iloc[:, 1:].to_numpy()
    
    return np.sum((actual_array - pred1_array)**2) / len(actual_array)

actual_error(df_actual_path, df_pred_path)

In [ ]:
def train_linear_reg(city):
    
    p_i, p_o = get_city_trajectories(city=city, split="train", normalized=True)
    train_combined = np.concatenate([p_i, p_o], axis=1)
    p_test_i, p_test_o = get_city_trajectories(city=city, split="test", normalized=True)

    def preprocess_data(inp, x_or_y=0):

        final_feature = inp[:, :50, x_or_y]
        final_target = inp[:, 50, x_or_y]
        for i in range(1, 60):
            temp_x = inp[:, i:50+i, x_or_y]
            temp_y = inp[:, 50+i, x_or_y]
            final_feature = np.concatenate([final_feature, temp_x], axis=0)
            final_target = np.concatenate([final_target, temp_y], axis=0)

        return final_feature, final_target

    x_feature, x_target = preprocess_data(train_combined, 0)
    y_feature, y_target = preprocess_data(train_combined, 1)

    reg = LinearRegression().fit(x_feature, x_target)
    reg1 = LinearRegression().fit(y_feature, y_target)
    
    print("training process for " + city + " is done")
    
    def predict_regressor(model, inp, pred_len, x_or_y=0):

        result = []
        curr = inp[:, x_or_y]
        pred = model.predict(curr.reshape(1, -1))[0]
        result.append(pred)
        for i in range(1, pred_len):
            curr = np.append(curr[1:], pred)
            pred = model.predict(curr.reshape(1, -1))[0]
            result.append(pred)

        return result
    
    palo_df = pd.DataFrame()
    
    print("starts generating csv file for " + city)
    
    for i in range(len(p_test_i)):

        pred_x = predict_regressor(reg, p_test_i[i], 60, 0)
        pred_y = predict_regressor(reg1, p_test_i[i], 60, 1)
        
        temp_dict = {}
        for j in range(0, 120, 2):
            col_x = 'v' + str(j)
            col_y = 'v' + str(j+1)
            row = j // 2
            temp_dict[col_x] = pred_x[row]
            temp_dict[col_y] = pred_y[row]

        temp_idx_name = str(i) + "_" + city
        temp_final = pd.DataFrame(temp_dict, index=[temp_idx_name])
        palo_df = pd.concat([palo_df, temp_final])
    
    palo_df.reset_index(level=0).rename(columns={"index": "ID"}).to_csv("model3_{}.csv".format(city), index=False)
    
    print("Prediction for " + city + " is done")
    
    return palo_df

In [ ]:
cities = ["austin", "miami", "pittsburgh", "dearborn", "washington-dc", "palo-alto"]
for city in cities[:-1]:
    
    train_linear_reg(city)


In [ ]:
a = pd.read_csv("./model3_austin.csv")
b = pd.read_csv("./model3_miami.csv")
c = pd.read_csv("./model3_pittsburgh.csv")
d = pd.read_csv("./model3_dearborn.csv")
e = pd.read_csv("./model3_washington-dc.csv")
f = pd.read_csv("./model3_palo_df.csv")
final = pd.concat([a, b, c, d, e, f], axis=0)

In [ ]:
final.to_csv("model3_submission.csv", index=False)

In [ ]:
pd.read_csv("model3_submission.csv")